#### Imports/setup

In [ ]:
from parameters.config_imports import *

In [ ]:
def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out

#### Input: Geo IDs

In [ ]:
# example_ids
GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids


#### Fetch plots and make a feature collection

In [ ]:
roi = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

### Whisp it

In [ ]:
df = whisp_stats_as_df(roi)

### Display it

In [ ]:
df

#### Export it to CSV file

In [ ]:
df.to_csv(path_or_buf=out_csv,header=True)

print ("output csv: ", out_csv)


##### Calculate EUDR risk category

In [ ]:
# Replace with the name of your CSV file
csv_file =out_directory/'whisp_output_table.csv'

# import as a dataframe
df_w_stats = pd.read_csv(csv_file)


df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )


##### Display table with risk columns

In [ ]:
df_w_risk

##### Export table with risk columns to csv 

In [ ]:
output_file=out_directory/'whisp_output_table_w_geo_id_w_risk.csv'

df_w_risk.to_csv(output_file)
